In [1]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms


# (80, 80)にResizeして(64, 64)にCenterCropしてTensorにする
img_data = ImageFolder('./oxford-102/', transform=transforms.Compose([transforms.Resize(80), transforms.CenterCrop(64), transforms.ToTensor()]))
batch_size = 64
img_loader = DataLoader(img_data, batch_size=batch_size, shuffle=True)

In [2]:
from torch import nn


nz = 100
ngf = 32


# 画像を生成するネットワーク
# (100, 1, 1)のでたらめな画像から(3, 64, 64)の画像を作る
# ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1)
# ConvTranspose2dによって画像サイズはこのように変化する out_size = (in_size -1) * stride - 2 * padding + kernel_size + output_padding この式によって出力したい画像サイズを調節すること
# 活性化関数の選択やバッチノーマリゼーションの使用はDCGANの元論文で提案された設定を採用してる
class GNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf, 3, 4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, x):
        out = self.main(x)
        return out

In [3]:
ndf = 32


# 画像を判断するネットワーク
# 畳み込み演算を繰り返して(3, 64, 64)の画像を一次元のスカラーに変換する
# 5回の畳込み演算で(3, 64, 64)が(1, 1, 1)になる
# squeeze: (A, 1, B, 1)というshapeを(A, B)に変換する、今回の場合はネットワークの入出力が(batch_size, channel, height, width)で出力が(batch_size, 1, 1, 1)となるので、出力を(batch_size)のみのスカラーにする
class DNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, stride=1, padding=0, bias=False)
        )

    def forward(self, x):
        out = self.main(x)
        return out.squeeze()

In [4]:
import torch
from torch import optim
from torch.autograd import Variable as V


d = DNet()
g = GNet()

# Adamのパラメータは元論文の提案値を採用、慣性項は0.5に設定されているのが特徴
opt_d = optim.Adam(d.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_g = optim.Adam(g.parameters(), lr=0.0002, betas=(0.5, 0.999))
# 実ラベル
ones = V(torch.ones(batch_size))
# 偽ラベル
zeros = V(torch.zeros(batch_size))
# 実偽の2値問題を扱うので損失関数はシグモイド関数を作用させたクロスエントロピー
loss_f = nn.BCEWithLogitsLoss()
fixed_z = V(torch.randn(batch_size, nz, 1, 1))

In [5]:
from statistics import mean


# 1. 偽画像を乱数と生成モデルから作る
# 2. 偽画像を識別モデルで判別する
# 3. 実画像を識別モデルで判別する
# 4. 偽画像のラベルが実として生成モデルのパラメータを更新
# 5. 実画像のラベルが実、偽画像のラベルが偽として判別モデルのパラメータを更新
def train_dcgan(g, d, opt_g, opt_d, loader):
    log_loss_g = []
    log_loss_d = []
    for real_img, _ in tqdm(loader):
        # 1.
        batch_len = len(real_img)
        # 偽画像を乱数と生成モデルから作る
        z = torch.randn(batch_len, nz, 1, 1)
        fake_img = g(V(z))
        # あとで使用するので偽画像の値のみ取り出しておく
        fake_img_tensor = fake_img.data
        
        # 2. 4.
        # 偽画像に対する生成モデルの評価関数を計算する
        out = d(fake_img)
        loss_g = loss_f(out, ones[:batch_len])
        log_loss_g.append(loss_g.data[0])
        # ネットワークの計算グラフ（計算過程）が生成モデルと識別モデル両方に依存しているので、両者とも勾配をクリアしてから、微分の計算とパラメータ更新を行う
        d.zero_grad()
        g.zero_grad()
        loss_g.backward()
        opt_g.step()
        
        # 2. 3. 5.
        # 実際の画像に対する識別モデルの評価関数を計算
        real_out = d(V(real_img))
        loss_d_real = loss_f(real_out, ones[:batch_len])
        # pytorchでは同じVariableを含んだ計算グラフに対して2回backwardを行うことができないので、Variableを保存してあったTensorから作り直し、ムダな計算を省く
        fake_img = V(fake_img_tensor)
        # 偽画像に対する識別モデルの評価関数の計算
        fake_out = d(fake_img)
        loss_d_fake = loss_f(fake_out, zeros[:batch_len])
        # 実偽の評価関数の合計値
        loss_d = loss_d_real + loss_d_fake
        log_loss_d.append(loss_d.data[0])
        # 識別モデルの微分計算とパラメータ更新
        d.zero_grad()
        g.zero_grad()
        loss_d.backward()
        opt_d.step()
    print(mean(log_loss_g), mean(log_loss_d), flush=True)
    return mean(log_loss_g), mean(log_loss_d)

In [6]:
from torchvision.utils import save_image
from tqdm import tqdm


for epoch in range(40):
    train_dcgan(g, d, opt_g, opt_d, img_loader)
    if epoch % 10 == 0:
        # 生成モデルパラメータの保存
        torch.save(
            g.state_dict(),
            './oxford-102-gen/g_{:03d}.prm'.format(epoch),
            pickle_protocol=4
        )
        # 識別モデルパラメータの保存
        torch.save(
            d.state_dict(),
            './oxford-102-gen/d_{:03d}.prm'.format(epoch),
            pickle_protocol=4
        )
        # モニタリング用zから生成した画像を保存
        generated_img = g(fixed_z).data
        save_image(
            generated_img,
            './oxford-102-gen/{:03d}.jpg'.format(epoch)
        )

100%|██████████| 128/128 [02:35<00:00,  1.21s/it]

2.4917652839794755 0.3224084503017366



100%|██████████| 128/128 [02:32<00:00,  1.19s/it]

3.083204898110125 0.40757381457660813



100%|██████████| 128/128 [02:31<00:00,  1.19s/it]

...

1.9400607170537114 0.6444734148681164



100%|██████████| 128/128 [02:43<00:00,  1.28s/it]

1.8831403388176113 0.621060281060636



100%|██████████| 128/128 [02:44<00:00,  1.28s/it]

1.8587512171361595 0.6316140000708401
